In [60]:
from allocate import *

fisher_info = load_json('/root/autodl-tmp/data.json')

bits = [2,4,8]
target_bits = 4.3
layers_num = len(fisher_info)*len(fisher_info[0])

allocation_strategy = Greedy_allocation_list(bits, target_bits, layers_num)
res = get_bits_list(fisher_info,allocation_strategy)

print(res)

None


In [51]:
def sort_FI_layer(data):
    from collections import defaultdict
    values_dict = defaultdict(list)
    for block in data:
        for key, value in block.items():
            values_dict[key].append(value)

    rank_dict = {}
    for key, values in values_dict.items():
        sorted_values = sorted(values, reverse=True)
        ranks = {v: i + 1 for i, v in enumerate(sorted_values)}
        rank_dict[key] = [ranks[v] for v in values]
    return rank_dict

def sort_FI_in_all(data):
    from collections import defaultdict

    all_values = []
    for index, block in enumerate(data):
        for key, value in block.items():
            all_values.append((f"{index}_{key}", value)) 

    sorted_values = sorted(all_values, key=lambda x: x[1], reverse=False)
    rank_dict = {}
    for rank, (combined_key, _) in enumerate(sorted_values, start=1):
        index, key = combined_key.split('_', 1)  
        rank_dict[(int(index), key)] = rank  

    result = defaultdict(dict)
    for (index, key), rank in rank_dict.items():
        result[index][key] = rank

    return dict(result)

In [57]:
# 判断当前排序的位宽结果
def judge_bits(rank,allocation_strategy):
    total_nums = 0
    for bit,nums in allocation_strategy.items():
        total_nums += nums
        if rank < total_nums or rank == total_nums:
            return bit

def get_bits_list(fisher_info, strategy):
    sorted_FI = sort_FI_in_all(fisher_info)
    res = []
    for index,layer_FI_rank in sorted_FI.items():
        layers_bit = {}
        for layer_name,FI_rank in layer_FI_rank.items():
            bit = judge_bits(FI_rank,strategy)
            layers_bit[layer_name] = bit
        res.append(layers_bit)
    return res

res = get_bits_list(fisher_info, allocation_strategy)
print(res)

[{'self_attn.q_proj.weight': 2, 'mlp.gate_proj.weight': 2, 'mlp.down_proj.weight': 2, 'mlp.up_proj.weight': 2, 'self_attn.k_proj.weight': 2, 'self_attn.o_proj.weight': 2, 'self_attn.v_proj.weight': 2}, {'self_attn.q_proj.weight': 2, 'self_attn.k_proj.weight': 2, 'mlp.down_proj.weight': 2, 'mlp.up_proj.weight': 2, 'mlp.gate_proj.weight': 2, 'self_attn.v_proj.weight': 2, 'self_attn.o_proj.weight': 2}, {'mlp.down_proj.weight': 2, 'self_attn.q_proj.weight': 2, 'mlp.up_proj.weight': 2, 'mlp.gate_proj.weight': 2, 'self_attn.k_proj.weight': 2, 'self_attn.v_proj.weight': 2, 'self_attn.o_proj.weight': 2}, {'mlp.down_proj.weight': 2, 'mlp.up_proj.weight': 2, 'mlp.gate_proj.weight': 2, 'self_attn.q_proj.weight': 2, 'self_attn.k_proj.weight': 2, 'self_attn.v_proj.weight': 2, 'self_attn.o_proj.weight': 2}, {'mlp.down_proj.weight': 2, 'mlp.up_proj.weight': 2, 'mlp.gate_proj.weight': 2, 'self_attn.k_proj.weight': 2, 'self_attn.q_proj.weight': 2, 'self_attn.o_proj.weight': 2, 'self_attn.v_proj.weight'